In [52]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-04-30'
end_date = '2023-05-31'
store_id = '985'

query = f"""
select qfo.source_store_id,
       sum(qfo.cnt_wares_pos_detail)                                                             as "Кол-во товаров в чеках", -- "Количество товаров в чеке"
       sum(qfo.order_sum_with_vat)                                                               as "Выручка, тг с НДС",
       sum(qfo.order_sum_without_vat)                                                            as "Выручка, тг без НДС",
       sum(qfo.is_sale_order)                                                                    as "Количество чеков"
from dwh_data.qs_fact_order qfo
where date(qfo.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
        and qfo."source_store_id"::int >= 0
group by qfo.source_store_id
     union all
select   qfog.source_store_id,
         sum(qfog.count_wares)                            as quantity,
         sum(qfog.sum_sale_vat)                           as order_sum_with_vat,
         sum(qfog.sum_sale_no_vat)                        as order_sum_without_vat,
         sum(qfog.is_sale_order)                          as sale_order_cnt
from dwh_data.qs_fact_order_gross qfog
where date(qfog.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
      and qfog."source_store_id"::int >= 0
group by qfog.source_store_id
order by source_store_id;

"""

cur.execute(query)

print('Executed')

df = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [21]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-05-01'
end_date = '2023-06-15'
store_id = '985'

query = f"""
SELECT
      q.source_store_id::int                                                                    as source_store_id,
      Sum(q."sum_order_wares_vat_nr")                                                           as "Оборот, тг с НДС",
      Sum(q."sum_order_wares_without_vat_nr")                                                   as "Оборот, тг без НДС",
      Sum(q."cost_price_base_t")                                                                as "Себестоимость",
      Sum(q."cost_price_base_t") -Sum(q."cost_price_base_t")*
              (Sum(q."sum_order_wares_vat_nr") -Sum(q."sum_order_wares_without_vat_nr") )
              /decode(Sum(q."sum_order_wares_vat_nr"),0,null,Sum(q."sum_order_wares_vat_nr"))   as "Бухгалтерская себестоимость"
   FROM "dwh_data"."qs_fact_sales" q
   WHERE 1=1
  --and to_char(q.order_date,'DD.MM.YYYY') in ($(vDatesList))
   and  date(q.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
    and q."source_store_id"::int >= 0
   GROUP BY
      "source_store_id"
 union all
    SELECT
          sa.source_store_id::int                          as source_store_id,
          Sum("sum_sale")                                  as sum_order_with_vat,
          Sum(sa."sum_sale_no_vat")                        as sum_order_without_vat,
          Sum("sum_coming")                                as cost_price_with_vat ,
          Sum(sa."sum_coming_no_vat")                      as cost_price_without_vat
    FROM "dwh_data"."qs_fact_sales_gross" sa
    left join dwh_data.dim_store ds on ds.source_store_id = sa.source_store_id and current_date between ds.datestart and ds.dateend
    WHERE 1=1
   and  date(sa.order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
   and sa.source_store_id::int >= 0
    GROUP BY
        sa.source_store_id
    order by source_store_id;
"""

cur.execute(query)

print('Executed')

df1 = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [53]:
df.columns = ["Номер филиала", "Количество товаров в чеке", "Выручка, тг с НДС", "Выручка, тг без НДС", "Количество чеков"]
df

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,1160,6267009.92,5595544.58,47
1,4,1323651,1123343574.97,1002975175.50,169456
2,96,1467450,1337514658.00,1194201535.28,185292
3,96,362,1949488.76,1740614.95,16
4,102,300090,237329551.00,211900535.53,58198
...,...,...,...,...,...
285,2889,22454,13032280.00,11635858.71,6966
286,2970,2,0.00,0.00,2
287,2975,33592,20023547.00,17877998.41,10315
288,2989,44,1098237.00,980568.75,2


In [60]:
f = pd.read_csv('C:\\Users\\Abdykarim.D\\Desktop\\_select_qfo_source_store_id_sum_qfo_cnt_wares_pos_detail_as_Кол__202306211451.csv', encoding='windows-1251')
f
f.to_excel('C:\\Users\\Abdykarim.D\\Desktop\\erkerj.xlsx')

In [23]:
df1.columns = ["Номер филиала", "Оборот, тг с НДС", "Оборот, тг без НДС", "Себестоимость", "Бухгалтерская себестоимость"]
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,8987483.89400000000000000000000000000000087048...,8024539.19107142857142857142857142857142934864...,8593171.85090479773005556049483369675492954888...,7684053.29693873995706942935351687665152178207...
1,4,1633145468.21799999962469530000000000000000000...,1458156954.87486042290866372242857142857140657...,1504233117.25796728777859541690649738526442186...,1343057323.65416571553309330032166449553000808...
2,96,1949684645.93300000052077686000000000000000000...,1740786555.81324757540100748499999999999998002...,1787603355.98602486573632914811483779718284170...,1596071393.24728094089950150457989576734433222...
3,96,3849105.54800000000000000000000000000000266135...,3436701.38214285714285714285714285714285951907...,3747761.13857188407111952111557085183239691980...,3352156.94660605170635671528175968913606867840...
4,102,346796020.477999999973712240000000000000000000...,309639303.998172913654113176428571428571429279...,333287066.806987338588984466657602689126500259...,297577738.220484649007542722424923615637765515...
...,...,...,...,...,...
287,2889,20039711.6450000000000000000000000000000000000...,17892599.6830325735614304885714285714285707496...,15778232.0632444797125517575970409619297825430...,14087707.1993229554709832805880332626640121006...
288,2970,12385575.4350000000000000000000000000000000000...,11058549.4955349329342430457142857142857145401...,9588780.80342445167338501291989664082687338562...,8561411.43162836979437871154723413694191636441...
289,2975,29380770.4619999999853364000000000000000000000...,26232732.3739249158335383200000000000000008596...,24249911.3052757459248426509488403356875552899...,21651625.3100127729747750565801113597098761960...
290,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [24]:
df

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,1160,6267009.92,5595544.58,47
1,4,1323651,1123343574.97,1002975175.50,169456
2,96,1467450,1337514658.00,1194201535.28,185292
3,96,362,1949488.76,1740614.95,16
4,102,300090,237329551.00,211900535.53,58198
...,...,...,...,...,...
285,2889,22454,13032280.00,11635858.71,6966
286,2970,2,0.00,0.00,2
287,2975,33592,20023547.00,17877998.41,10315
288,2989,44,1098237.00,980568.75,2


In [25]:
df1

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,8987483.89400000000000000000000000000000087048...,8024539.19107142857142857142857142857142934864...,8593171.85090479773005556049483369675492954888...,7684053.29693873995706942935351687665152178207...
1,4,1633145468.21799999962469530000000000000000000...,1458156954.87486042290866372242857142857140657...,1504233117.25796728777859541690649738526442186...,1343057323.65416571553309330032166449553000808...
2,96,1949684645.93300000052077686000000000000000000...,1740786555.81324757540100748499999999999998002...,1787603355.98602486573632914811483779718284170...,1596071393.24728094089950150457989576734433222...
3,96,3849105.54800000000000000000000000000000266135...,3436701.38214285714285714285714285714285951907...,3747761.13857188407111952111557085183239691980...,3352156.94660605170635671528175968913606867840...
4,102,346796020.477999999973712240000000000000000000...,309639303.998172913654113176428571428571429279...,333287066.806987338588984466657602689126500259...,297577738.220484649007542722424923615637765515...
...,...,...,...,...,...
287,2889,20039711.6450000000000000000000000000000000000...,17892599.6830325735614304885714285714285707496...,15778232.0632444797125517575970409619297825430...,14087707.1993229554709832805880332626640121006...
288,2970,12385575.4350000000000000000000000000000000000...,11058549.4955349329342430457142857142857145401...,9588780.80342445167338501291989664082687338562...,8561411.43162836979437871154723413694191636441...
289,2975,29380770.4619999999853364000000000000000000000...,26232732.3739249158335383200000000000000008596...,24249911.3052757459248426509488403356875552899...,21651625.3100127729747750565801113597098761960...
290,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [57]:
df2 = pd.concat([df, df1], axis=1, join="inner", )
df2

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,2403,750860,474786325.00,423915219.23,181488,1565,1309497514.29499999946546316000000000000000000...,1169212276.88472935491710285728571428571428891...,1148464763.53901503337992920543712890327875073...,1025430813.30113314135818596849972789421788661...
1,1910,2625617,1573860568.00,1405262727.31,578648,1025,6153259757.35000000211326519200000000000000000...,5495715979.96709153787495820065079365079353821...,5693624527.60421975658266636449627666868369481...,5085197851.90457610412386561083295922186574137...
2,2382,469751,329411547.00,294121417.22,108255,91,53373.5999999999999999999999999999999999850032000,47654.999999999999999999999999999999999986610,57835.384265163960621091960160,51638.7359510392505545463930
3,542,110534304,66658064518.79,59515889206.51,11928517,1025,109750.999999999999999999999999999999999983520...,97991.9642857142857142857142857142857142710000...,93876.0000000000000000000000000000000000337600...,83817.8571428571428571428571428571428571730000...
4,2127,1338400,985781735.00,880179493.61,221316,2124,1115753983.26700000012180444000000000000000000...,996228170.378317702875119025142857142857121595...,995110734.245989134137233874219610105205641515...,888508901.576087150027854375791366269014533139...
...,...,...,...,...,...,...,...,...,...,...
379,1353,1511222,1236393509.30,1104169116.46,83404,2781,203065176.350000000157498840000000000000000000...,181314129.376768841375975125999999999999995464...,148346411.922336405420234705152503240177775742...,132456490.114810109907654884781923161500889169...
380,280,62877,1692630902.97000000000086400000000000000000000...,1522719636.93857142857220000000000000000000000...,5307,2059,1148454930.22900000000000000000000000000000000...,1025406037.21152229133944614999999999999998257...,1027762811.37517332926893709649326631067513726...,917645232.621752940969936995611027552303011877...
381,969,1957,8602619.45,7681629.02,170,1612,3367923555.12199999953046578000000000000000000...,3007240173.83858090612058660257142857142855023...,2999856296.19020749901736663113013943911781633...,2678590598.03363410950080206776137752051244685...
382,2433,1,23032.10,20564.38,1,1439,1.99999999900000048000000000000000000000000000...,1.78571428482142900000000000000000000000000000...,4557.67494689861111909197041089997560713100416...,4069.35263115947421347497358116069250636696800...


In [26]:
df2 = df1.copy()
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0, 1], dtype='int64')
Оборот, тг с НДС               1642132952.111999999624695300
Оборот, тг без НДС             1466181494.065931851480092294
Себестоимость                  1512826289.108872085508650977
Бухгалтерская себестоимость    1350741376.951104455490162730
dtype: object
Index([2, 3], dtype='int64')
Оборот, тг с НДС               1953533751.481000000520776860
Оборот, тг без НДС             1744223257.195390432543864628
Себестоимость                  1791351117.124596749807448669
Бухгалтерская себестоимость    1599423550.193886992605858220
dtype: object
Index([4, 5], dtype='int64')
Оборот, тг с НДС               350025098.4779999999737122400
Оборот, тг без НДС             312522409.3553157707969703193
Себестоимость                  336285598.9480395385889844667
Бухгалтерская себестоимость    300258644.4717391632932570081
dtype: object
Index([6, 7], dtype='int64')
Оборот, тг с НДС               872878692.0150000001003202200
Оборот, тг без НДС             779354431.07615217

In [27]:
df2

,Номер филиала,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,4,1642132952.111999999624695300,1466181494.065931851480092294,1512826289.108872085508650977,1350741376.951104455490162730
1,4,1633145468.21799999962469530000000000000000000...,1458156954.87486042290866372242857142857140657...,1504233117.25796728777859541690649738526442186...,1343057323.65416571553309330032166449553000808...
2,96,1953533751.481000000520776860,1744223257.195390432543864628,1791351117.124596749807448669,1599423550.193886992605858220
3,96,3849105.54800000000000000000000000000000266135...,3436701.38214285714285714285714285714285951907...,3747761.13857188407111952111557085183239691980...,3352156.94660605170635671528175968913606867840...
4,102,350025098.4779999999737122400,312522409.3553157707969703193,336285598.9480395385889844667,300258644.4717391632932570081
...,...,...,...,...,...
287,2889,20039711.6450000000000000000000000000000000000...,17892599.6830325735614304885714285714285707496...,15778232.0632444797125517575970409619297825430...,14087707.1993229554709832805880332626640121006...
288,2970,12385575.4350000000000000000000000000000000000...,11058549.4955349329342430457142857142857145401...,9588780.80342445167338501291989664082687338562...,8561411.43162836979437871154723413694191636441...
289,2975,29380770.4619999999853364000000000000000000000...,26232732.3739249158335383200000000000000008596...,24249911.3052757459248426509488403356875552899...,21651625.3100127729747750565801113597098761960...
290,2989,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...,1098236.99999999948480000000000000000000000000...,980568.749999999540000000000000000000000000000...


In [29]:
len(df1['Номер филиала'].unique())

247

In [31]:
len(df2)

292

In [32]:
df3 = df.copy()
for i in df3['Номер филиала'].unique():
    print(i, df3[df3['Номер филиала'] == i].index)
    ids = df3[df3['Номер филиала'] == i].index
    print(df3.loc[ids, df3.columns[1:]].sum())
    df3.loc[ids[0], df3.columns[1:]] = df3.loc[ids, df3.columns[1:]].sum()
    try:
        df3 = df3.drop([ids[1]])
    except:
        pass

4 Index([0, 1], dtype='int64')
Количество товаров в чеке          1324811
Выручка, тг с НДС            1129610584.89
Выручка, тг без НДС          1008570720.08
Количество чеков                    169503
dtype: object
96 Index([2, 3], dtype='int64')
Количество товаров в чеке          1467812
Выручка, тг с НДС            1339464146.76
Выручка, тг без НДС          1195942150.23
Количество чеков                    185308
dtype: object
102 Index([4, 5], dtype='int64')
Количество товаров в чеке          300259
Выручка, тг с НДС            240077375.00
Выручка, тг без НДС          214353949.81
Количество чеков                    58213
dtype: object
105 Index([6, 7], dtype='int64')
Количество товаров в чеке          712262
Выручка, тг с НДС            578917761.74
Выручка, тг без НДС          516888589.54
Количество чеков                   104683
dtype: object
116 Index([8, 9], dtype='int64')
Количество товаров в чеке                           375356
Выручка, тг с НДС            490099440.1100

In [33]:
df3 = df3.reset_index()
df3 = df3.drop(['index'], axis=1)
df3

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,1324811,1129610584.89,1008570720.08,169503
1,96,1467812,1339464146.76,1195942150.23,185308
2,102,300259,240077375.00,214353949.81,58213
3,105,712262,578917761.74,516888589.54,104683
4,116,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...
242,2889,22454,13032280.00,11635858.71,6966
243,2970,2,0.00,0.00,2
244,2975,33592,20023547.00,17877998.41,10315
245,2989,44,1098237.00,980568.75,2


In [34]:
for i in df3['Номер филиала']:
    print(i, end=', ')

4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1165, 1187, 1205, 1208, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1615, 1638, 1698, 1718, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1883, 1886, 1898, 1907, 1910, 1913, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2024, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193

In [35]:
inds = []
not_inds = []
for i in range(len(df1)):
    if df1.iloc[i].equals(df2.iloc[i]):
        inds.append(i)
    else:
        not_inds.append(i)

In [43]:
len(df3)

247

In [44]:
# inds1 = []
# not_inds1 = []
# for i in range(len(df)):
#     print(i)
#     try:
#         if df.iloc[i].equals(df3.iloc[i]):
#             inds1.append(i)
#         else:
#             not_inds1.append(i)
#     except:
#         print(df.iloc[i], df3.iloc[i])

In [45]:
df3

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков
0,4,1324811,1129610584.89,1008570720.08,169503
1,96,1467812,1339464146.76,1195942150.23,185308
2,102,300259,240077375.00,214353949.81,58213
3,105,712262,578917761.74,516888589.54,104683
4,116,375356,490099440.1100000000000000000,437588555.2457142857142857143,48852
...,...,...,...,...,...
242,2889,22454,13032280.00,11635858.71,6966
243,2970,2,0.00,0.00,2
244,2975,33592,20023547.00,17877998.41,10315
245,2989,44,1098237.00,980568.75,2


In [ ]:
for i in df2['Номер филиала'].unique():
    print(df2[df2['Номер филиала'] == i].index)
    ids = df2[df2['Номер филиала'] == i].index
    print(df2.loc[ids, df2.columns[1:]].sum())
    df2.loc[ids[0], df2.columns[1:]] = df2.loc[ids, df2.columns[1:]].sum()

Index([0], dtype='int64')
Количество товаров в чеке                                                  11783
Выручка, тг с НДС                                                     7525031.00
Выручка, тг без НДС                                                   6718734.66
Количество чеков                                                            2589
Оборот, тг с НДС               82133802811.8219999545656379960000000000000000...
Оборот, тг без НДС             73347493095.2100944447412386556031746031742836...
Себестоимость                  57494025081.6455697745459859511759702966979874...
Бухгалтерская себестоимость    51343569435.7603851262871592390984197815968943...
dtype: object
Index([1, 2], dtype='int64')
Количество товаров в чеке                          132801427
Выручка, тг с НДС              82365149978.53000000008417696
Выручка, тг без НДС            73521297372.96154761912279914
Количество чеков                                    13344863
Оборот, тг с НДС               57053341

In [ ]:
df2.loc[[1, 2], df2.columns[1:]].sum()

Количество товаров в чеке                          132875366
Выручка, тг с НДС              82596248661.23000000016835392
Выручка, тг без НДС            73727900744.35309523824559828
Количество чеков                                    13348164
Оборот, тг с НДС               578058850.0393632132343045094
Оборот, тг без НДС             516389234.9938097469773039315
Себестоимость                  554449338.2460351552897313918
Бухгалтерская себестоимость    498882079.7768801754627076399
dtype: object

In [ ]:
pd.concat([df, df1], axis=1, join="inner")

,Номер филиала,Количество товаров в чеке,"Выручка, тг с НДС","Выручка, тг без НДС",Количество чеков,"Оборот, тг с НДС","Оборот, тг без НДС",Себестоимость,Бухгалтерская себестоимость
0,985,11783,7525031.00,6718734.66,2589,82133802811.8219999545656379960000000000000000...,73347493095.2100944447412386556031746031742836...,57494025081.6455697745459859511759702966979874...,51343569435.7603851262871592390984197815968943...
1,4,132727488,82134051295.83,73314694001.57,13341562,563007982.993363213274250389408395786066240549...,502950960.845595461489249895701940682003587788...,540527736.599755072089731391755183500350775607...,486452078.306987244210403794698508562867111343...
2,4,73939,231098682.700000000084176960000000000000000067...,206603371.391547619122799142857142857142857202...,3301,7525433.52299999998002706000000000000000000000...,6719137.07410714274402701785714285714285733855...,6960800.82314004159999999999999999999999999200...,6215000.73494646562615192260393837401195928088...


In [46]:
for i in df3['Номер филиала']:
    print(i, end=', ')

4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1165, 1187, 1205, 1208, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1615, 1638, 1698, 1718, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1883, 1886, 1898, 1907, 1910, 1913, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2024, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193

In [47]:
s = [4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1065, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1187, 1205, 1208, 1332, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1612, 1615, 1638, 1698, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1879, 1883, 1898, 1907, 1910, 1913, 1918, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2019, 2024, 2026, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2066, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193, 2196, 2241, 2261, 2262, 2281, 2290, 2294, 2299, 2300, 2303, 2304, 2305, 2306, 2315, 2320, 2323, 2326, 2341, 2344, 2347, 2350, 2353, 2361, 2364, 2367, 2381, 2382, 2383, 2384, 2401, 2402, 2403, 2412, 2421, 2422, 2423, 2424, 2433, 2436, 2439, 2442, 2445, 2482, 2485, 2488, 2491, 2494, 2521, 2524, 2541, 2563, 2583, 2601, 2604, 2641, 2643, 2655, 2661, 2684, 2702, 2723, 2741, 2745, 2748, 2751, 2763, 2766, 2781, 2785, 2801, 2809, 2810, 2829, 2832, 2849, 2889, 2970, 2975, 2989, 3012, 3033]
s1 = [4, 96, 102, 105, 116, 140, 197, 220, 280, 320, 360, 401, 421, 441, 444, 461, 501, 521, 541, 542, 561, 601, 602, 631, 632, 633, 634, 635, 641, 681, 701, 721, 725, 767, 785, 805, 832, 845, 849, 852, 866, 885, 889, 893, 897, 901, 925, 945, 949, 953, 957, 961, 965, 969, 985, 989, 993, 1005, 1009, 1025, 1029, 1033, 1069, 1105, 1135, 1136, 1137, 1138, 1146, 1151, 1152, 1165, 1187, 1205, 1208, 1373, 1375, 1412, 1432, 1435, 1439, 1452, 1552, 1555, 1556, 1557, 1565, 1568, 1571, 1574, 1578, 1581, 1584, 1592, 1595, 1615, 1638, 1698, 1718, 1738, 1778, 1782, 1798, 1818, 1838, 1841, 1858, 1861, 1864, 1867, 1870, 1883, 1886, 1898, 1907, 1910, 1913, 1938, 1941, 1944, 1947, 1950, 1955, 1956, 1959, 1960, 1963, 1966, 1970, 1975, 1979, 1980, 1998, 2018, 2024, 2027, 2029, 2032, 2041, 2044, 2047, 2050, 2059, 2061, 2065, 2071, 2074, 2078, 2081, 2083, 2087, 2091, 2098, 2099, 2104, 2110, 2118, 2121, 2124, 2127, 2138, 2141, 2142, 2147, 2150, 2151, 2152, 2155, 2158, 2161, 2162, 2167, 2181, 2182, 2183, 2184, 2193, 2196, 2241, 2261, 2262, 2281, 2290, 2294, 2299, 2300, 2303, 2304, 2305, 2306, 2315, 2320, 2323, 2326, 2341, 2344, 2347, 2350, 2353, 2361, 2364, 2367, 2381, 2382, 2383, 2384, 2401, 2402, 2403, 2412, 2421, 2422, 2423, 2433, 2436, 2439, 2501, 2521, 2524, 2541, 2563, 2582, 2601, 2604, 2641, 2643, 2655, 2661, 2684, 2702, 2723, 2741, 2745, 2751, 2763, 2781, 2785, 2801, 2810, 2829, 2849, 2869, 2889, 2970, 2975, 2989, 3012]
len(s)

263

In [64]:
for i in s:
    print(i)

4
96
102
105
116
140
197
220
280
320
360
401
421
441
444
461
501
521
541
542
561
601
602
631
632
633
634
635
641
681
701
721
725
767
785
805
832
845
849
852
866
885
889
893
897
901
925
945
949
953
957
961
965
969
985
989
993
1005
1009
1025
1029
1033
1065
1069
1105
1135
1136
1137
1138
1146
1151
1152
1187
1205
1208
1332
1373
1375
1412
1432
1435
1439
1452
1552
1555
1556
1557
1565
1568
1571
1574
1578
1581
1584
1592
1595
1612
1615
1638
1698
1738
1778
1782
1798
1818
1838
1841
1858
1861
1864
1867
1870
1879
1883
1898
1907
1910
1913
1918
1938
1941
1944
1947
1950
1955
1956
1959
1960
1963
1966
1970
1975
1979
1980
1998
2018
2019
2024
2026
2027
2029
2032
2041
2044
2047
2050
2059
2061
2065
2066
2071
2074
2078
2081
2083
2087
2091
2098
2099
2104
2110
2118
2121
2124
2127
2138
2141
2142
2147
2150
2151
2152
2155
2158
2161
2162
2167
2181
2182
2183
2184
2193
2196
2241
2261
2262
2281
2290
2294
2299
2300
2303
2304
2305
2306
2315
2320
2323
2326
2341
2344
2347
2350
2353
2361
2364
2367
2381
2382
2383
2384
2401


In [48]:
import numpy as np
len(np.unique(np.asarray(s)))

263

In [50]:
s = np.asarray(s)
s1 = np.asarray(s1)
print(np.setdiff1d(s1, s))

[1165 1718 1886 2501 2582 2869]


In [51]:
len(df3)

247